In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_727947/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.8)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_5_5,0.999629,0.705050,0.999495,0.999230,0.999396,0.000156,0.175095,0.000243,0.000362,0.000303,0.004730,0.012485,0.999227,0.012585,147.533018,331.891342,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_5_9,0.999629,0.704327,0.999446,0.999029,0.999279,0.000156,0.175525,0.000267,0.000457,0.000362,0.004853,0.012488,0.999227,0.012588,147.531964,331.890288,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_5_8,0.999628,0.704391,0.999454,0.999080,0.999307,0.000156,0.175487,0.000263,0.000432,0.000348,0.004835,0.012501,0.999225,0.012601,147.527837,331.886161,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_5_10,0.999628,0.704315,0.999441,0.999001,0.999263,0.000156,0.175532,0.000269,0.000470,0.000370,0.004854,0.012507,0.999224,0.012607,147.525961,331.884285,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_5_7,0.999627,0.704649,0.999465,0.999121,0.999331,0.000157,0.175333,0.000258,0.000413,0.000336,0.004841,0.012522,0.999223,0.012623,147.521043,331.879367,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.714541,0.485513,0.578479,0.644434,0.632728,0.119942,0.305422,0.135897,0.191298,0.163597,0.192172,0.346327,0.405294,0.349108,134.241493,318.599817,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.684795,0.454833,0.519667,0.605576,0.587979,0.132441,0.323634,0.154857,0.212204,0.183531,0.203414,0.363924,0.343323,0.366847,134.043242,318.401566,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.625389,0.429706,0.471230,0.532139,0.526100,0.157401,0.338551,0.170473,0.251714,0.211094,0.210706,0.396738,0.219561,0.399925,133.697912,318.056236,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.160865,0.061324,0.048991,0.200243,0.172864,0.352582,0.557239,0.306601,0.430277,0.368439,0.364804,0.593786,-0.748198,0.598556,132.084945,316.443269,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
